In [2]:
import requests
from io import BytesIO

from PIL import Image
from vllm import LLM, SamplingParams

llm = LLM(model="allenai/Molmo-7B-D-0924", trust_remote_code=True)


/home/ubuntu/.cache/pypoetry/virtualenvs/orign-Sz0VPFy2-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-26 20:38:29,175	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 10-26 20:38:29 config.py:1664] Downcasting torch.float32 to torch.float16.
INFO 10-26 20:38:33 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='allenai/Molmo-7B-D-0924', speculative_config=None, tokenizer='allenai/Molmo-7B-D-0924', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=allenai/Molmo-7B-D-0924, num_scheduler_ste

Loading safetensors checkpoint shards:   0% Completed | 0/7 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  14% Completed | 1/7 [00:00<00:02,  2.21it/s]
Loading safetensors checkpoint shards:  29% Completed | 2/7 [00:01<00:04,  1.21it/s]
Loading safetensors checkpoint shards:  43% Completed | 3/7 [00:02<00:04,  1.06s/it]
Loading safetensors checkpoint shards:  57% Completed | 4/7 [00:04<00:03,  1.17s/it]
Loading safetensors checkpoint shards:  71% Completed | 5/7 [00:05<00:02,  1.26s/it]
Loading safetensors checkpoint shards:  86% Completed | 6/7 [00:06<00:01,  1.12s/it]
Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:07<00:00,  1.13s/it]
Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:07<00:00,  1.09s/it]



INFO 10-26 20:38:43 model_runner.py:1067] Loading model weights took 14.9975 GB


/home/ubuntu/.cache/pypoetry/virtualenvs/orign-Sz0VPFy2-py3.12/lib/python3.12/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/ubuntu/.cache/pypoetry/virtualenvs/orign-Sz0VPFy2-py3.12/lib/python3.12/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


INFO 10-26 20:38:56 gpu_executor.py:122] # GPU blocks: 26351, # CPU blocks: 4681
INFO 10-26 20:38:56 gpu_executor.py:126] Maximum concurrency for 4096 tokens per request: 102.93x
INFO 10-26 20:39:00 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-26 20:39:00 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-26 20:39:11 model_runner.py:1523] Graph capturing finished in 11 secs.


In [5]:
def open_image_from_url(url: str) -> Image.Image:
    """
    Opens an image from a given URL.

    Parameters:
    url (str): The URL of the image.

    Returns:
    Image.Image: The opened PIL Image object.
    """
    response = requests.get(url)
    response.raise_for_status()  # Check if the request was successful
    return Image.open(BytesIO(response.content))


image_1 = open_image_from_url("https://picsum.photos/id/237/536/354",)
image_2 = open_image_from_url("https://picsum.photos/id/238/536/354")

outputs = llm.generate(
    prompts=[
        {
            "prompt": "Whats the content of this image?",
            "multi_modal_data": {"image_0": image_1},
        },
        {
            "prompt": "Whats the color of this image?",
            "multi_modal_data": {"image_1": image_2},
        },
        {
            "prompt": "Whats the color of this image?",
            "multi_modal_data": {"image_1": [image_1, image_2]},
        }
    ],
    sampling_params=SamplingParams(max_tokens=100, n=4),
)

for o in outputs:
    print("output: ", o, "----\n")
    generated_text = o.outputs[0].text
    print(generated_text)

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.14s/it, est. speed input: 10.52 toks/s, output: 313.80 toks/s]

output:  RequestOutput(request_id=3, prompt='Whats the content of this image?', prompt_token_ids=tensor([151643,   2657,     25,  29756,    279,   2213,    315,    419,   2168,
            30,  21388,     25], dtype=torch.int32), encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' The image contains text in a decorative font. It displays the letter "A" repeated twice. The text appears to be in a stylized or ornate typeface, which gives it a visually ornamental quality. The repetition of the letter "A" creates an interesting visual pattern or design element.', token_ids=(576, 2168, 5610, 1467, 304, 264, 45436, 3301, 13, 1084, 18689, 279, 6524, 330, 32, 1, 11504, 10917, 13, 576, 1467, 7952, 311, 387, 304, 264, 48204, 1506, 476, 39445, 349, 943, 1564, 11, 892, 6696, 432, 264, 42295, 39445, 43086, 4271, 13, 576, 53415, 315, 279, 6524, 330, 32, 1, 11450, 458, 7040, 9124, 5383, 476, 2884, 2392, 13, 151643), cumulative_logprob=No